In [10]:
import datetime

from google.cloud import bigquery
import pandas as pd
import pytz
from google.oauth2 import service_account
import numpy as np

In [11]:
credentials = service_account.Credentials.from_service_account_file('pax-10-366616-49c701642ebe.json')

In [12]:
client = bigquery.Client(credentials=credentials)

In [13]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='34.75.252.143',
                         user='root',
                         password='Suman#007',
                         db='sales')

# create cursor
cursor=connection.cursor()

In [14]:
df_order_details=pd.read_sql(f'''select * from order_details''',connection)


C:\Users\suman\projeevt1\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
df_consumer_master=pd.read_sql(f'''select * from customer_master''',connection)

C:\Users\suman\projeevt1\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:

df_order_items=pd.read_sql(f'''select * from order_items''',connection)

C:\Users\suman\projeevt1\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
df_product_master=pd.read_sql(f'''select * from product_master''',connection)

C:\Users\suman\projeevt1\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
df_order_details.head(20)

,orderid,customerid,order_status_update_timestamp,order_status
0,1,50,2022-11-20 13:29:19,Delivered
1,1,50,2022-11-19 18:14:02,InProgress
2,1,50,2022-11-19 16:48:54,Received
3,2,990,2022-03-31 07:47:27,Delivered
4,2,990,2022-03-30 15:18:23,InProgress
5,2,990,2022-03-30 02:00:36,Received
6,3,627,2022-07-22 21:09:31,Delivered
7,3,627,2022-07-22 20:15:51,InProgress
8,3,627,2022-07-22 06:23:57,Received
9,4,916,2022-09-08 11:21:46,Delivered


In [19]:
def dim_order(df_order_details):
    g = df_order_details
    #g = df_order_details.groupby("orderid").tail(1)
    x = g[["orderid","order_status_update_timestamp","order_status"]]
    dim_order = x.reset_index()
    del dim_order["index"]
    return dim_order

In [20]:
dim_order = dim_order(df_order_details)

In [21]:
dim_order.head()

,orderid,order_status_update_timestamp,order_status
0,1,2022-11-20 13:29:19,Delivered
1,1,2022-11-19 18:14:02,InProgress
2,1,2022-11-19 16:48:54,Received
3,2,2022-03-31 07:47:27,Delivered
4,2,2022-03-30 15:18:23,InProgress


In [22]:
tableRef2 = client.dataset("star_schema").table("dim_order")
client.load_table_from_dataframe(dim_order,tableRef2)

LoadJob<project=pax-10-366616, location=US, id=b4d928fb-4578-40a8-8fb5-c3cce465fc39>

In [23]:
def insert_dim_customer(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_customer =df_consumer_master[["customerid","name","address_id"]]
    dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
    dim_customer["end_date"] = np.nan
    dim_customer["end_date"]=pd.to_datetime(dim_customer["end_date"])
    return dim_customer

In [24]:
dim_customer = insert_dim_customer(df_consumer_master)

<ipython-input-23-5860bea26682>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
<ipython-input-23-5860bea26682>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["end_date"] = np.nan
<ipython-input-23-5860bea26682>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [25]:
dim_customer.head()

,customerid,name,address_id,start_date,end_date
0,1,Aaryahi Khurana,1,2022-08-23,NaT
1,2,Vaibhav Kade,2,2022-04-28,NaT
2,3,Trisha Yadav,3,2022-01-13,NaT
3,4,Romil Taneja,4,2022-06-23,NaT
4,5,Pihu Devan,5,2022-07-31,NaT


In [26]:
tableRef4 = client.dataset("star_schema").table("dim_customer")
client.load_table_from_dataframe(dim_customer,tableRef4)

LoadJob<project=pax-10-366616, location=US, id=1845ee66-f642-4f27-b839-0ce4c188ac5c>

In [27]:
def insert_dim_address(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_address = df_consumer_master[["address_id","address","city","state","pincode"]]
    return dim_address

In [28]:
dim_address = insert_dim_address(df_consumer_master)

In [29]:

dim_address.dtypes

address_id     int64
address       object
city          object
state         object
pincode        int64
dtype: object

In [30]:
tableRef1 = client.dataset("star_schema").table("dim_address")
client.load_table_from_dataframe(dim_address,tableRef1)

LoadJob<project=pax-10-366616, location=US, id=2b5ad87e-6d00-43d4-b5f5-fe2df4b34923>

In [31]:
def insert_dim_product(df_product_master):
    dim_product = df_product_master
    dim_product["start_date"]=np.nan
    dim_product["start_date"]=pd.to_datetime(dim_product["start_date"])
    dim_product["end_date"]=np.nan
    dim_product["end_date"]=pd.to_datetime(dim_product["end_date"])
    return dim_product

In [32]:
dim_product=insert_dim_product(df_product_master)

In [33]:
tableRef5 = client.dataset("star_schema").table("dim_product")
client.load_table_from_dataframe(dim_product,tableRef5)

LoadJob<project=pax-10-366616, location=US, id=fb41aeff-a7c3-4db2-bd1c-aef62682e2ac>

In [34]:
def insert_f_order_details(df_order_details,df_order_items):
    f_order_details = pd.merge(df_order_details.groupby("orderid").tail(1), df_order_items, how='inner')[["orderid","order_status_update_timestamp","productid","quantity"]]
    f_order_details.columns = ["orderid","order_delivery_timestamp","productid","quantity"]
    return f_order_details

In [35]:
f_order_details = insert_f_order_details(df_order_details,df_order_items)


In [36]:
f_order_details.head()


,orderid,order_delivery_timestamp,productid,quantity
0,1,2022-11-19 16:48:54,22,5
1,1,2022-11-19 16:48:54,37,5
2,2,2022-03-30 02:00:36,2,1
3,2,2022-03-30 02:00:36,9,1
4,2,2022-03-30 02:00:36,23,5


In [37]:
tableRef6 = client.dataset("star_schema").table("f_order_details")
client.load_table_from_dataframe(f_order_details,tableRef6)

LoadJob<project=pax-10-366616, location=US, id=ab9c9c32-20c7-46f7-894e-681726156cc9>

In [38]:
def insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items):
    x=df_order_details.groupby("orderid").head(1)[["customerid","orderid","order_status_update_timestamp"]].reset_index()
    del x["index"]
    y=df_order_details.groupby("orderid").tail(1)["order_status_update_timestamp"].reset_index()
    del y["index"]     
    fact_daily_orders = pd.concat([x,y],axis=1)
    fact_daily_orders.columns = ["customerid","orderid","order_received_timestamp","order_delivery_timestamp"]
    l=[]
    for i in fact_daily_orders["customerid"]:
             l.append(int(df_consumer_master.where(df_consumer_master["customerid"]==i).dropna()["pincode"]))
            # l is pincode column
    fact_daily_orders["pincode"] = l
        
    
    k=[]
    m=[]
    for i in df_order_items["productid"]:
        m=list(df_product_master.where(df_product_master["productid"]==i)["rate"].dropna() * df_order_items.iloc[i,2])
        k.append(m[0])
    df_order_items["Total"]=k    
    k = df_order_items.groupby('orderid')["Total"].sum()
    k1 = df_order_items.groupby('orderid')["quantity"].sum()
    k=k.tolist()
    fact_daily_orders["order_amount"]=k
    fact_daily_orders["item_count"]=list(k1)
    fact_daily_orders["order_delivery_time_seconds"] =fact_daily_orders["order_delivery_timestamp"] - fact_daily_orders["order_received_timestamp"]
    return fact_daily_orders

In [39]:
fact_daily_orders = insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items)

In [40]:
fact_daily_orders.head(10)

,customerid,orderid,order_received_timestamp,order_delivery_timestamp,pincode,order_amount,item_count,order_delivery_time_seconds
0,50,1,2022-11-20 13:29:19,2022-11-19 16:48:54,111463,7873.0,10,-1 days +03:19:35
1,990,2,2022-03-31 07:47:27,2022-03-30 02:00:36,843590,14161.0,11,-2 days +18:13:09
2,627,3,2022-07-22 21:09:31,2022-07-22 06:23:57,640651,7583.0,13,-1 days +09:14:26
3,916,4,2022-09-08 11:21:46,2022-09-07 11:36:35,363894,8989.0,4,-1 days +00:14:49
4,24,5,2022-03-11 10:15:07,2022-03-10 00:00:55,775124,1731.0,5,-2 days +13:45:48
5,107,6,2022-08-26 01:00:05,2022-08-25 00:28:33,719259,7461.0,10,-2 days +23:28:28
6,262,7,2022-03-25 15:27:15,2022-03-24 06:45:41,274472,12814.0,7,-2 days +15:18:26
7,731,8,2022-03-17 15:14:51,2022-03-16 12:33:26,169999,2774.0,1,-2 days +21:18:35
8,437,9,2022-04-24 06:23:38,2022-04-23 10:38:35,359107,16063.0,13,-1 days +04:14:57
9,938,10,2022-02-20 00:40:31,2022-02-19 08:05:21,17280,1497.0,2,-1 days +07:24:50


In [41]:
fact_daily_orders.dtypes

customerid                               int64
orderid                                  int64
order_received_timestamp        datetime64[ns]
order_delivery_timestamp        datetime64[ns]
pincode                                  int64
order_amount                           float64
item_count                               int64
order_delivery_time_seconds    timedelta64[ns]
dtype: object

In [42]:
tableRef3 = client.dataset("star_schema").table("fact_daily_orders")
client.load_table_from_dataframe(fact_daily_orders,tableRef3)

C:\Users\suman\projeevt1\env\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:552: UserWarning: Pyarrow could not determine the type of columns: order_delivery_time_seconds.
  warnings.warn(


LoadJob<project=pax-10-366616, location=US, id=bea73496-e3bc-4692-8436-2227ad269fb6>